In [18]:
# Projede kullanacağım kütüphaneleri yüklüyorum

import pandas as pd   # veri işleme için pandas
import numpy as np    # sayısal işlemler için numpy

# Dosya işlemleri ve sistem fonksiyonları için
import os 

import traceback  # hata durumunda ayrıntılı bilgi almak için

# Mesafe hesaplama için gerekli matematik fonksiyonlarını import ediyorum
# (haversine fonksiyonu yazarken kullanacağım)
from math import radians, cos, sin, asin, sqrt

# En yakın noktayı daha hızlı bulmak için KDTree yapısını kullanıyorum
from scipy.spatial import KDTree

# Görselleştirme araçları
import matplotlib.pyplot as plt   # grafik çizmek için
import seaborn as sns             # istatistiksel görselleştirme için

# Harita üzerinde analiz yapabilmek için folium kütüphanesini kullanıyorum
import folium
from folium.plugins import MarkerCluster  # haritada noktaları gruplayarak göstermek için

# Jupyter ortamında çıktı alabilmek ve haritayı gömebilmek için
from IPython.display import display, clear_output, IFrame


In [19]:
# --- ROUTING HELPER (OSRM) ---
import requests

# OSRM (Open Source Routing Machine) üzerinden rota almak için fonksiyon yazıyorum
def get_route_osrm(start_lat, start_lon, end_lat, end_lon, profile='walking'):
    base = "https://router.project-osrm.org/route/v1"   # OSRM'in temel URL'si
    coords = f"{start_lon},{start_lat};{end_lon},{end_lat}"  # koordinatları [lon,lat] formatında yazmam gerekiyor
    url = f"{base}/{profile}/{coords}"  # istek atacağım URL'yi tamamlıyorum

    # Rotanın nasıl döneceğini belirleyen parametreleri ekliyorum
    params = {"overview": "full", "geometries": "geojson", "steps": "true"}

    try:
        # OSRM'e istek gönderiyorum
        r = requests.get(url, params=params, timeout=8)
        r.raise_for_status()  # HTTP hatası varsa burada fırlatılır

        # Dönen yanıtı JSON formatına çeviriyorum
        j = r.json()
        routes = j.get("routes")

        # Eğer rota bulunamadıysa None döndürüyorum
        if not routes:
            return None

        r0 = routes[0]  # ilk rotayı alıyorum

        # Koordinatları GeoJSON formatından alıyorum
        geom = r0.get("geometry", {}).get("coordinates", [])

        # OSRM [lon, lat] döndürür → ben [lat, lon] olacak şekilde çeviriyorum
        coords_latlon = [[c[1], c[0]] for c in geom]

        # Fonksiyon geriye rotanın koordinatlarını, mesafesini, süresini ve adımlarını döndürüyor
        return {
            "coords": coords_latlon,
            "distance": r0.get("distance"),
            "duration": r0.get("duration"),
            "steps": r0.get("legs", [])[0].get("steps", []) if r0.get("legs") else []
        }

    except Exception as e:
        # İstek başarısız olursa uyarı veriyorum
        print("OSRM isteği başarısız:", e)
        return None


In [20]:
def haversine_m(lat1, lon1, lat2, lon2):
    """
    İki nokta arasındaki mesafeyi metre cinsinden hesaplıyorum (Haversine formülü).
    lat1, lon1 : tek bir noktanın koordinatları (float)
    lat2, lon2 : bir nokta ya da birden fazla nokta (array, liste veya pd.Series olabilir)
    Dönüş      : mesafeyi metre cinsinden döndürüyorum (numpy array veya float)
    """
    # Derece cinsinden gelen koordinatları radyana çeviriyorum
    lat1_r = np.radians(lat1)
    lon1_r = np.radians(lon1)
    lat2_r = np.radians(np.asarray(lat2, dtype=float))
    lon2_r = np.radians(np.asarray(lon2, dtype=float))

    # Enlem ve boylam farklarını hesaplıyorum
    dlat = lat2_r - lat1_r
    dlon = lon2_r - lon1_r

    # Haversine formülünü uyguluyorum
    a = np.sin(dlat/2.0)**2 + np.cos(lat1_r) * np.cos(lat2_r) * np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    # Dünya yarıçapını (6371000 m) çarpıp mesafeyi metre cinsinden döndürüyorum
    return 6371000.0 * c


def estimate_walk_time_minutes(dist_m, speed_kmh=5.0):
    """Mesafeye göre tahmini yürüme süresini dakika cinsinden hesaplıyorum. 
    Varsayılan hız: 5 km/saat"""
    if dist_m is None:
        return None
    # Hızı metre/dakika cinsine çevirip mesafeyi böldüğümde dakikayı buluyorum
    return dist_m / (speed_kmh * 1000.0 / 60.0)


In [21]:
# Dolum merkezleri verisini yüklüyorum
dolum = pd.read_csv(
    "dolummerkezi.csv",        # kullandığım veri dosyası
    encoding="ISO-8859-9",     # Türkçe karakterlerin bozulmaması için uygun encoding seçiyorum
    engine="python"            # parser olarak python kullanıyorum (bazı özel durumlarda daha esnek)
)

print(dolum.shape)             # veri setinde kaç satır ve sütun olduğunu kontrol ediyorum
dolum.head()                   # veri setinin ilk 5 satırını ön izleme yapıyorum


(11356, 5)


,ï»¿_id,COUNTY_NAME,ADRESS,LONGITUDE,LATITUDE
0,1,19 MAYIS MH.,34210 13 NO:1,"28,585371","41,024945"
1,2,ADALAR,Recep KoÃ§ Recep KoÃ§ Recep KoÃ§ Cad. No:50/1,29.0,41.0
2,3,ADALAR,BÃ¼yÃ¼kada RECEP KOÃ CAD. No:25,"29,129939","40,87458"
3,4,ADALAR,Heybeliada RÄ±htÄ±m Cd. No:1/B,"29,101164","40,876861"
4,5,ADALAR,Heybeliada Heybeliada Heybeliada Mah. AyyÄ±ldÄ...,29.0,41.0


In [22]:
# Rota verilerini yüklüyorum
routes = pd.read_csv("routes.csv")   # routes.csv dosyasını okudum

print(routes.shape)   # dosyada kaç satır ve sütun olduğunu kontrol ediyorum
routes.head()         # ilk 5 satırı gösterip ön izleme yapıyorum


(8047, 7)


,route_id,agency_id,route_short_name,route_long_name,route_type,route_desc,route_code
0,1,1.0,1,KADIKÖY - KİRAZLITEPE,3.0,NaN,1_D_D0
1,2,1.0,1,KADIKÖY - KİRAZLITEPE,3.0,SAATLER FERAH CADDESİNDEN GİDER.,1_D_D2784
2,3,1.0,1,KİRAZLITEPE - KADIKÖY,3.0,NaN,1_G_D0
3,4,1.0,1,KİRAZLITEPE - KADIKÖY,3.0,SAATLER FERAH CADDESİNDEN GİDER.,1_G_D2783
4,5,1.0,1,KİRAZLITEPE - KADIKÖY,3.0,NaN,1_G_D2900


In [23]:
# Toplu taşıma durakları verisini yüklüyorum
stops = pd.read_csv(
    "stops.csv", 
    encoding="ISO-8859-9",         # Türkçe karakterler için doğru encoding
    usecols=[0,1,2,3,4,5,6],       # sadece ilk 7 sütunu alıyorum, gereksiz kolonları okumuyorum
    engine="python"                # python parser daha esnek, özel durumlarda hata ihtimalini azaltıyor
)

print(stops.shape)   # dosyanın boyutunu (satır, sütun) kontrol ediyorum
stops.head()         # ilk 5 satırı gösterip hızlıca göz atıyorum


(14848, 7)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type
0,292235,100001,RIFAT ILGAZ CADDESİ,direction: AVCILAR,41.0191700005564,28.684353,0.0
1,292236,100002,RIFAT ILGAZ CADDESİ,direction: BOĞAZKÖY,41.0190470005563,28.684660,0.0
2,292237,100011,UZUNYAYLA,direction: AVCILAR,41.0104540005549,28.688629,0.0
3,292238,100012,UZUNYAYLA,direction: BOĞAZKÖY,41.010675000555,28.688751,0.0
4,292239,100021,OKTAY RIFAT CADDESİ,direction: AVCILAR,41.0045240005537,28.695276,0.0


In [24]:
# Duraklardaki hareket saatlerini yüklüyorum
# Burada sadece ihtiyacım olan kolonları alıyorum: trip_id, arrival_time, departure_time, stop_id
stop_times = pd.read_csv(
    "stop_times.csv",
    usecols=["trip_id", "arrival_time", "departure_time", "stop_id"],  # gereksiz kolonları okumuyorum
    dtype={   # bellek kullanımı ve veri tipi hatalarını önlemek için tipleri baştan belirliyorum
        "trip_id": "int32",
        "arrival_time": "string",
        "departure_time": "string",
        "stop_id": "int32"
    }
)

print(stop_times.shape)   # dosyada kaç satır ve sütun olduğunu kontrol ediyorum
stop_times.head()         # ilk 5 satırı görüntülüyerek hızlı bir kontrol yapıyorum


(1048575, 4)


,trip_id,stop_id,arrival_time,departure_time
0,323167998,510964,06:40:00,06:40:00
1,323167998,510801,<NA>,<NA>
2,323167998,510676,<NA>,<NA>
3,323167998,510643,<NA>,<NA>
4,323167998,510659,<NA>,<NA>


In [25]:
# Dolum merkezi verisini tekrar yüklüyorum
# Önceki okuma sorun çıkarsa diye encoding'i UTF-8 yaptım
dolum = pd.read_csv("dolummerkezi.csv", encoding="utf-8")  

print(dolum.shape)   # veri setinde kaç satır ve sütun olduğunu kontrol ediyorum
dolum.head()         # ilk 5 satırı görüntüleyip veri yapısını hızlıca gözden geçiriyorum


(11356, 5)


,_id,COUNTY_NAME,ADRESS,LONGITUDE,LATITUDE
0,1,19 MAYIS MH.,34210 13 NO:1,"28,585371","41,024945"
1,2,ADALAR,Recep Koç Recep Koç Recep Koç Cad. No:50/1,29.0,41.0
2,3,ADALAR,Büyükada RECEP KOÇ CAD. No:25,"29,129939","40,87458"
3,4,ADALAR,Heybeliada Rıhtım Cd. No:1/B,"29,101164","40,876861"
4,5,ADALAR,Heybeliada Heybeliada Heybeliada Mah. Ayyıldız...,29.0,41.0


In [26]:
# Dolum merkezi verisini temizliyorum

# 1️⃣ Kolon isimlerindeki BOM (gizli karakterler) ve boşlukları siliyorum
dolum.rename(columns=lambda x: x.strip(), inplace=True)

# 2️⃣ LATITUDE ve LONGITUDE sütunlarını güvenli şekilde float'a çeviriyorum
def safe_float(x):
    # Önce string'e çeviriyorum
    x = str(x)
    # Virgül varsa noktaya çeviriyorum (örn: "41,01" → "41.01")
    x = x.replace(',', '.')
    # Geçerli float değilse NaN döndürüyorum
    try:
        return float(x)
    except:
        return np.nan

# Kolonlara uyguluyorum
dolum['LATITUDE'] = dolum['LATITUDE'].apply(safe_float)
dolum['LONGITUDE'] = dolum['LONGITUDE'].apply(safe_float)

# 3️⃣ Geçersiz koordinatları (NaN olanları) düşürüyorum
before = dolum.shape[0]
dolum = dolum.dropna(subset=['LATITUDE','LONGITUDE']).reset_index(drop=True)
after = dolum.shape[0]
print(f"Geçersiz koordinatlar düşürüldü: {before - after} / Kalan satır: {after}")

# 4️⃣ Kontrol amaçlı ilk 5 koordinatı görüntülüyorum
dolum[['LATITUDE','LONGITUDE']].head()


Geçersiz koordinatlar düşürüldü: 169 / Kalan satır: 11187


,LATITUDE,LONGITUDE
0,41.024945,28.585371
1,41.000000,29.000000
2,40.874580,29.129939
3,40.876861,29.101164
4,41.000000,29.000000


In [27]:
# --- 2) Durak ve dolum koordinatlarını NumPy array'e çeviriyorum, önce hatalı değerleri temizliyorum ---

# 1️⃣ Stops (duraklar) için
stops['stop_lat'] = pd.to_numeric(stops['stop_lat'], errors='coerce')  # geçersiz değer varsa NaN yap
stops['stop_lon'] = pd.to_numeric(stops['stop_lon'], errors='coerce')
stops = stops.dropna(subset=['stop_lat','stop_lon']).reset_index(drop=True)  # NaN olanları çıkarıyorum

# NumPy array'e çeviriyorum (KDTree veya mesafe hesapları için)
stop_coords = stops[['stop_lat', 'stop_lon']].to_numpy()

# 2️⃣ Dolum merkezleri için aynı işlemi yapıyorum
dolum['LATITUDE'] = pd.to_numeric(dolum['LATITUDE'], errors='coerce')
dolum['LONGITUDE'] = pd.to_numeric(dolum['LONGITUDE'], errors='coerce')
dolum = dolum.dropna(subset=['LATITUDE','LONGITUDE']).reset_index(drop=True)

# NumPy array'e çeviriyorum
center_coords = dolum[['LATITUDE','LONGITUDE']].to_numpy()


In [28]:
# --- 1) Haversine fonksiyonu (metre cinsinden) ---
def haversine(lat1, lon1, lat2, lon2):
    """
    İki koordinat arasındaki mesafeyi metre cinsinden hesaplıyorum.
    lat1, lon1 : birinci nokta
    lat2, lon2 : ikinci nokta
    Dönüş     : metre cinsinden mesafe
    """
    # Dereceleri radyana çeviriyorum
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Enlem ve boylam farklarını hesaplıyorum
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formülünü uyguluyorum
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))

    # Dünya yarıçapını metre olarak alıyorum
    r = 6371000  
    return c * r   # iki nokta arasındaki mesafe (metre)


In [29]:
# Durak ve dolum merkezi koordinatlarını NumPy array'e çeviriyorum
# KDTree veya mesafe hesaplamalarında daha hızlı işlem yapabilmek için

# Duraklar
stop_coords = stops[['stop_lat', 'stop_lon']].astype(float).to_numpy()  

# Dolum merkezleri
center_coords = dolum[['LATITUDE','LONGITUDE']].to_numpy()


In [30]:
# KDTree ile hızlandırılmış en yakın dolum merkezi arama

# 1️⃣ Dolum merkezi koordinatlarını radyana çevirip KDTree oluşturuyorum
# KDTree, mesafe sorgularını çok hızlı yapmamı sağlıyor
tree = KDTree(np.radians(center_coords))  

# 2️⃣ Her durak için en yakın dolum merkezini buluyorum
# stop_coords da radyana çevriliyor, k=1 → sadece en yakın merkez
distances, indices = tree.query(np.radians(stop_coords), k=1)


In [31]:
# KDTree ile bulunan en yakın dolum merkezi indekslerini gerçek mesafeye (metre) çeviriyorum

# KDTree yaklaşık olarak en yakın noktayı buluyor ama mesafe tam olarak metre cinsinden değil
# Bu yüzden Haversine formülünü kullanıyorum
dist_m = [
    haversine(
        stop_coords[i][0], stop_coords[i][1],                   # durak koordinatları
        center_coords[indices[i]][0], center_coords[indices[i]][1]  # en yakın dolum merkezi koordinatları
    )
    for i in range(len(stop_coords))  # tüm duraklar için
]


In [32]:
# --- 5) Sonuçları durak tablosuna ekliyorum ---

# Her durak için en yakın dolum merkezi indeksini ekliyorum
stops['nearest_center_index'] = indices

# Her durak ile en yakın dolum merkezi arasındaki mesafeyi metre cinsinden ekliyorum
stops['distance_m'] = dist_m


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
from folium.plugins import MarkerCluster

# Output alanı
output = widgets.Output()

# Textbox ve buton
textbox = widgets.Text(placeholder='İlçe veya mahalle girin')
button = widgets.Button(description="Haritayı Göster")

# Butonu ve textbox'u ekrana bas
display(textbox, button, output)


Text(value='', placeholder='İlçe veya mahalle girin')

Button(description='Haritayı Göster', style=ButtonStyle())

Output()

In [34]:
def on_button_click(b):  # butona basınca çalışan fonksiyon
    with output:
        clear_output(wait=True)  # önceki çıktıyı siliyoruz

        # Kullanıcı inputunu al ve temizle
        user_input = textbox.value.strip().upper()  # büyük harfe çeviriyoruz
        if user_input == "":
            print("Lütfen bir ilçe veya mahalle adı girin.")
            return

        # Dolum merkezlerini temizle ve eşleşme için hazırla
        dolum['COUNTY_NAME'] = dolum['COUNTY_NAME'].astype(str).str.strip().str.upper()
        filtered_dolum = dolum[dolum['COUNTY_NAME'].str.contains(user_input, na=False, regex=False)].copy()

        if filtered_dolum.empty:
            print("Girilen bölgede dolum merkezi bulunamadı.")
            return

        # Koordinatları sayıya çevir, hatalıları NaN yap
        filtered_dolum['LATITUDE'] = pd.to_numeric(filtered_dolum['LATITUDE'], errors='coerce')
        filtered_dolum['LONGITUDE'] = pd.to_numeric(filtered_dolum['LONGITUDE'], errors='coerce')
        before_drop = len(filtered_dolum)
        filtered_dolum = filtered_dolum.dropna(subset=['LATITUDE', 'LONGITUDE']).copy()
        dropped = before_drop - len(filtered_dolum)
        if dropped > 0:
            print(f"⚠️ {dropped} kayıt koordinat eksikliğinden atıldı.")

        # Mantıksız koordinatları sil (-90..90 / -180..180)
        valid_mask = (
            (filtered_dolum['LATITUDE'] >= -90) & (filtered_dolum['LATITUDE'] <= 90) &
            (filtered_dolum['LONGITUDE'] >= -180) & (filtered_dolum['LONGITUDE'] <= 180)
        )
        if not valid_mask.all():
            problem_count = (~valid_mask).sum()
            print(f"⚠️ {problem_count} kayıt geçersiz koordinat aralığında; atılıyor.")
        filtered_dolum = filtered_dolum[valid_mask].copy()

        if filtered_dolum.empty:
            print("Girilen bölgede geçerli koordinata sahip dolum merkezi bulunamadı.")
            return

        # Harita merkezi
        center_lat = float(filtered_dolum['LATITUDE'].median())
        center_lon = float(filtered_dolum['LONGITUDE'].median())
        if np.isnan(center_lat) or np.isnan(center_lon):
            row0 = filtered_dolum.iloc[0]
            center_lat = float(row0['LATITUDE'])
            center_lon = float(row0['LONGITUDE'])

        # Boş harita oluştur
        m = folium.Map(location=[center_lat, center_lon], zoom_start=14)
        dolum_cluster = MarkerCluster(name='Dolum Merkezleri').add_to(m)

        walking_threshold_m = 800
        search_radius_m = 2000
        short_distance_threshold = 100
        match_count = 0

        # Her dolum merkezi için
        for idx, drow in filtered_dolum.iterrows():
            lat_d = float(drow['LATITUDE'])
            lon_d = float(drow['LONGITUDE'])

            # Yakındaki durakları seç
            deg_offset = search_radius_m / 111000.0
            min_lat, max_lat = lat_d - deg_offset, lat_d + deg_offset
            min_lon, max_lon = lon_d - deg_offset, lon_d + deg_offset
            candidate_stops = stops[
                (stops['stop_lat'] >= min_lat) & (stops['stop_lat'] <= max_lat) &
                (stops['stop_lon'] >= min_lon) & (stops['stop_lon'] <= max_lon)
            ].copy()

            best_stop = None
            best_dist = None
            if not candidate_stops.empty:
                dists = haversine_m(
                    lat_d, lon_d,
                    candidate_stops['stop_lat'].astype(float),
                    candidate_stops['stop_lon'].astype(float)
                )
                dists_arr = np.asarray(dists)
                argmin = int(np.argmin(dists_arr))
                best_dist = float(dists_arr[argmin])
                best_stop = candidate_stops.iloc[argmin]

            # Popup içeriği
            popup_lines = [f"<b>{drow.get('COUNTY_NAME', '')}</b>"]
            if best_stop is not None:
                stop_name = best_stop.get('stop_name', 'Bilinmeyen durak')
                walk_label = "Yürüme mesafesinde" if best_dist <= walking_threshold_m else "Yürüme mesafesinin dışında"
                walk_min = estimate_walk_time_minutes(best_dist)
                popup_lines.append(f"Önerilen durak: <b>{stop_name}</b>")
                popup_lines.append(f"Yürüme mesafesi: {best_dist:.0f} m (~{walk_min:.0f} dk)")
                popup_lines.append(f"<i>{walk_label}</i>")
                match_count += 1
            else:
                popup_lines.append("<i>Yakın durağı bulunamadı.</i>")

            popup_html = '<br>'.join(popup_lines)

            # Dolum marker
            folium.Marker(
                location=[lat_d, lon_d],
                popup=popup_html,
                tooltip=drow.get('COUNTY_NAME', ''),
                icon=folium.Icon(color='red', icon='tint', prefix='fa')
            ).add_to(dolum_cluster)

            # Durak marker + rota
            if best_stop is not None:
                s_lat = float(best_stop['stop_lat'])
                s_lon = float(best_stop['stop_lon'])
                folium.Marker(
                    location=[s_lat, s_lon],
                    popup=f"Önerilen Durak: {stop_name} — {best_dist:.0f} m",
                    icon=folium.Icon(color='green', icon='flag', prefix='fa')
                ).add_to(m)

                if best_dist <= short_distance_threshold:
                    folium.PolyLine(
                        locations=[[s_lat, s_lon], [lat_d, lon_d]],
                        weight=3, opacity=0.7,
                        tooltip=f"{stop_name} → {drow.get('COUNTY_NAME','')} (kısa mesafe, düz çizgi)"
                    ).add_to(m)
                else:
                    route = get_route_osrm(s_lat, s_lon, lat_d, lon_d, profile='walking')
                    if route and route.get('coords'):
                        folium.PolyLine(
                            locations=route['coords'],
                            weight=4, opacity=0.85,
                            tooltip=f"{stop_name} → {drow.get('COUNTY_NAME','')}"
                        ).add_to(m)
                    else:
                        folium.PolyLine(
                            locations=[[s_lat, s_lon], [lat_d, lon_d]],
                            weight=3, opacity=0.7,
                            tooltip=f"{stop_name} → {drow.get('COUNTY_NAME','')} (fallback düz çizgi)"
                        ).add_to(m)

        # Harita görünümü ve katman kontrolü
        try:
            lats = filtered_dolum['LATITUDE'].astype(float)
            lons = filtered_dolum['LONGITUDE'].astype(float)
            lat_buffer = (lats.max() - lats.min()) * 0.05 if (lats.max() - lats.min()) != 0 else 0.01
            lon_buffer = (lons.max() - lons.min()) * 0.05 if (lons.max() - lons.min()) != 0 else 0.01
            m.fit_bounds([[lats.min()-lat_buffer, lons.min()-lon_buffer], [lats.max()+lat_buffer, lons.max()+lon_buffer]])
        except:
            m.location = [center_lat, center_lon]

        folium.LayerControl().add_to(m)

        # Kaydet
        map_filename = "dolum_ve_duraklar_map.html"
        try:
            m.save(map_filename)
            print(f"✅ Harita başarıyla oluşturuldu ve kaydedildi: {map_filename}")
        except Exception as e:
            print("Harita kaydedilirken hata oluştu:", e)

        # Özet
        print(f"{len(filtered_dolum)} dolum merkezi bulundu. {match_count} tanesi için durak eşleşmesi yapıldı.")

        # --- Detaylı kontrol ---
        print(f"🔹 Kullanıcı inputu: '{user_input}'")
        print(f"🔹 Filtrelenen dolum merkezleri sayısı: {filtered_dolum.shape[0]}\n")
        if filtered_dolum.empty:
            print("⚠️ Hiç dolum merkezi eşleşmedi.")
        else:
            display(filtered_dolum[['COUNTY_NAME', 'LATITUDE', 'LONGITUDE']])
            n_invalid_coords = filtered_dolum[['LATITUDE', 'LONGITUDE']].isna().sum().sum()
            if n_invalid_coords > 0:
                print(f"⚠️ Dikkat: {n_invalid_coords} adet geçersiz koordinat var.")
            else:
                print("🔹 Tüm koordinatlar geçerli.")
                
button.on_click(on_button_click)

